In [ ]:
import numpy as np
from sympy import *
from scipy.spatial import Delaunay 
from mayavi import mlab
init_printing()

In [ ]:
rn = np.linspace(0, 5*np.pi)
theta_n = np.linspace(0, 2*np.pi)
R, Theta = np.meshgrid(rn, theta_n)
X = R*np.cos(Theta)
Y = R*np.sin(Theta)
W = np.cos(np.sqrt(X**2+Y**2))

In [ ]:
mlab.mesh(X, Y, W)
mlab.show()

In [ ]:
mlab.mesh(X, Y, W, scalars=1/(R+10), colormap='inferno')
mlab.show()

In [ ]:
x = [0, 1, 0, 0]
y = [0, 0, 0, 1]
z = [0, 0, 1, 0]
dreiecke = [[0,1,2], [0,1,3], [0,2,3],[1,2,3]]
mlab.triangular_mesh(x, y, z, dreiecke, colormap='viridis')
mlab.show()

In [ ]:
mlab.triangular_mesh(x, y, z, dreiecke, scalars=np.arange(4), colormap='hot')
# legt die Farben der Ecken fest
mlab.show()

In [ ]:
z = Symbol('z')

In [ ]:
v = z/(1+z**4)
pole = [complex(p.n()) for p in solveset(Eq(denom(v), 0))]

In [ ]:
pole

In [ ]:
epsilon = .07

def dreieck_gueltig(simplex, d):
    punkte = [d.points[nr] for nr in simplex]
    for p in punkte:
        z = p[0] + 1j*p[1]
        for pol in pole:
            if abs(pol-z) < .999*epsilon:
                return False
    return True

def plot_rational_curve():
    r = np.linspace(0, 2, 100)
    phi = np.linspace(0, 2*np.pi, 400)
    R, Phi = np.meshgrid(r, phi)
    Rf = R.flatten()
    Phi_f = Phi.flatten()
    Z = Rf*np.exp(1j*Phi_f)
    tn = np.linspace(0, 2*np.pi, 120)
    for pol in pole:
        z_neu = pol + epsilon*np.exp(1j*tn)
        Z = np.hstack([Z, z_neu])
    X = Z.real
    Y = Z.imag
    V = Z / (1+Z**4)
    W = abs(V)
    Theta = np.angle(V)
    d = Delaunay(np.array([X, Y]).T)
    simplices = [s for s in d.simplices if dreieck_gueltig(s, d)] 
    mlab.triangular_mesh(X, Y, W, simplices,
                         scalars=Theta, colormap='hsv')

In [ ]:
plot_rational_curve()
mlab.show()

In [ ]:
def plot_moebiusband():
    t = sympy.Symbol('t')
    s = sympy.Symbol('s')
    M = sympy.Matrix([ 3*sympy.cos(t) + s*sympy.sin(t/2),
                       3*sympy.sin(t),
                       s*sympy.cos(t/2)])
    sn = np.linspace(-1, 1)
    tn = np.linspace(0, 2*np.pi)
    S1, T1 = np.meshgrid(sn, tn)
    Sn, Tn = S1.T, T1.T
    fn = [sympy.lambdify((s,t), M[j], 'numpy') for j in range(3)]
    Xn = [fn[j](Sn, Tn) for j in range(3)]
    mlab.mesh(Xn[0], Xn[1], Xn[2], color=(1,0,0))
    return M

In [ ]:
def plot_normale(M):
    s = sympy.Symbol('s')
    t = sympy.Symbol('t')
    seele = M.subs(s, 0)
    Mt = seele.diff(t)
    Ms = M.diff(s)
    display(Mt, Ms)
    N = Mt.cross(Ms)
    display(N)
    N_ein = seele + s*N/N.norm()
    sn = np.linspace(0.05, 1)
    tn = np.linspace(0, 2*np.pi)
    S1, T1 = np.meshgrid(sn, tn)
    Sn, Tn = S1.T, T1.T
    Nn = [sympy.lambdify((s,t), N_ein[j], 'numpy') for j in range(3)]
    Xn = [Nn[j](Sn, Tn) for j in range(3)]
    mlab.mesh(Xn[0], Xn[1], Xn[2], color=(1,.8,0))

In [ ]:
test_triangular_mesh('hsv')
mlab.show()

In [ ]:
M = plot_moebiusband()
plot_normale(M)
mlab.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
?Delaunay

In [ ]:
r = np.linspace(1,3,10)
theta = np.linspace(0, 2*np.pi, 20)
R, Theta = np.meshgrid(r, theta)

In [ ]:
X = R*np.cos(Theta)
Y = R*np.sin(Theta)
plt.plot(X, Y, 'o')
plt.axis('equal')

In [ ]:
plt.plot(X.flatten(), Y.flatten(), '-')
plt.axis('equal');

In [ ]:
punkte = list(zip(X.flatten(), Y.flatten()))
dreiecke = Delaunay(punkte);

In [ ]:
d = dreiecke.simplices[0]
d

In [ ]:
def plot_strecke(p1, p2, polar):
    if polar:
        x = [p*np.cos(t) for p, t in [p1, p2]]
        y = [p*np.sin(t) for p, t in [p1, p2]]
    else:
        x = [p1[0], p2[0]]
        y = [p1[1], p2[1]]
    plt.plot(x, y, 'k')

def plot_dreieck(dreieck, dreiecke, polar=False):
    punkte = [dreiecke.points[nr] for nr in dreieck]
    for j in range(3):
        plot_strecke(punkte[j], punkte[(j+1)%3], polar)  

In [ ]:
plot_dreieck(dreiecke.simplices[0], dreiecke)

In [ ]:
for dreieck in dreiecke.simplices:
    plot_dreieck(dreieck, dreiecke)
    plt.axis('equal')

Delaunay trianguliert immer die konvexe Hülle

In [ ]:
d = Delaunay(list(zip(R.flatten(), Theta.flatten())))

In [ ]:
for dreieck in d.simplices:
    plot_dreieck(dreieck, d, polar=True)
    plt.axis('equal')